# Videos Pipeline

In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# load helper functions
%run -i "0. Functions_Clases Pipeline.py"
%run -i "Line.py"

# Load Camera calibration params
[ret, mtx, dist, rvecs, tvecs] = pickle.load(open( "pickle_data/camera_calibration_params.p", "rb" ) )


[NbConvertApp] Converting notebook 0. Functions_Clases Pipeline.ipynb to script
[NbConvertApp] Writing 19633 bytes to 0. Functions_Clases Pipeline.py


In [2]:
from time import time
def process_frame_profiling(img):
    
    ###### Resize image
    start = time()
    img = resizeImage(img)
    end = time()
    print(f'resizeImage took {end - start} seconds!')
    
    ###### Undistort image
    start = time()
    undistorted = undistort_image(img, mtx, dist)
    end = time()
    print(f'undistorted took {end - start} seconds!')
    
    ###### Color Enhancement
    start = time()
    imageCE = colorEnhancement(img)
    end = time()
    print(f'colorEnhancement took {end - start} seconds!')
    
   
    ###### GrayScale
    start = time()
    imageGray = grayscale(imageCE)
    end = time()
    print(f'grayscale took {end - start} seconds!') 

    ###### Gauss Smoothing
    start = time()
    imageGauss = gaussian_blur(imageGray,kernel_size=5)
    end = time()
    print(f'gaussian_blur took {end - start} seconds!') 
    
    #### Edge detection
    start = time()
    sbinary = sobel_thresh(imageGauss, sobel_kernel=5, x_thresh=[80,100], y_thresh=[40,100], mag_thresh=[50,255], dir_thresh=[100,200])
    end = time()
    print(f'sobel_thresh took {end - start} seconds!') 
    
    #### ROI
    start = time()
    ysize =sbinary.shape[0]
    xsize =sbinary.shape[1]

    ROI_upperWidth = 350 #Width of the upper horizontal straight in px
    ROI_upperHeight = 300 #Height of the upper horizontal straight from the bottom of the image in px
    ROI_lowerWidth = 1000 #Width of the lower horizontal straight in px
    ROI_lowerHeight = 50  #Height of the lower horizontal straight  from the bottom of the image in px      

    limitLL = ((xsize/2)-(ROI_lowerWidth/2),ysize-ROI_lowerHeight);
    limitLR = (xsize - ((xsize/2)-(ROI_lowerWidth/2)),ysize-ROI_lowerHeight);
    limitUL = ((xsize/2)-(ROI_upperWidth/2), ysize-ROI_upperHeight);
    limitUR = ((xsize/2)+(ROI_upperWidth/2), ysize-ROI_upperHeight);
    vertices = np.array([[limitLL,limitUL,limitUR , limitLR]], dtype=np.int32)
    
    imageROI = region_of_interest(sbinary,vertices)

    end = time()
    print(f'region_of_interest took {end - start} seconds!')
    
    #### Perspective transform
    start = time()
    warped_img,M, Minv = warp_image(imageROI, hwidth = 250 ,offset = 0, height = -600, overplotLines= False )
    end = time()
    print(f'warp_image took {end - start} seconds!')
    
    #### Find lines
    
    # Find x line poitns based on histogram values
    leftx_base, rightx_base  = find_lane_x_points(warped_img)
    
    # Update x base points
    lineLeft.updateXbase(leftx_base)
    lineRight.updateXbase(rightx_base)
    
    
    #Speed up coef with area search
    #Left line
    if lineLeft.missdetections > 0 or np.any((lineLeft.recent_poly_fits == 0)):
        ## Find lane pixels
        start = time()
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(warped_img, lineLeft.bestx, lineRight.bestx, showRectangles = False)
        end = time()
        print(f'find_lane_pixels took {end - start} seconds!')
        ## Update lane pixels
        lineLeft.updatePixels(leftx, lefty)   

        # Search blindly image
        start = time()
        coeffs_fit_L, lineDetectedL, left_fitx, ploty, img_line  = fit_polynomial(out_img, lineLeft.allx, lineLeft.ally, drawPoly = False)
        end = time()
        print(f'Search blindly image {end - start} seconds!')
        
    else:
        # Search based on coefs
        start = time()
        leftx, lefty, coeffs_fit_L, lineDetectedL, left_fitx, out_img = search_around_poly(warped_img, lineLeft)
        end = time()
        print(f'Search based on coefs {end - start} seconds!')
        lineLeft.updatePixels(leftx, lefty)  

    #Right line
    if lineRight.missdetections > 0 or np.any((lineRight.recent_poly_fits == 0)):
        ## Update lane pixels
        lineRight.updatePixels(rightx, righty)
        
        # Search blindly image
        coeffs_fit_R, lineDetectedR, right_fitx, ploty, img_line  = fit_polynomial(out_img, lineRight.allx, lineRight.ally, drawPoly = False)
    else:
        # Search based on coefs
        rightx, righty, coeffs_fit_R, lineDetectedR, right_fitx, out_img = search_around_poly(out_img, lineRight)   
        lineRight.updatePixels(rightx, righty)
        
           
    #Update line class instances

    lineLeft.updateCoeffsLine(lineDetectedL, coeffs_fit_L, left_fitx, lineLeft.poly_ploty,coefLimits=[0.01,1,100],movingAvg=5 )
    lineRight.updateCoeffsLine(lineDetectedR, coeffs_fit_R,right_fitx,lineRight.poly_ploty,coefLimits=[0.01,1,100],movingAvg=5 )
    
        
    
    ### Unwarp images
    
    #color_warp = np.zeros_like(out_img).astype(np.uint8)
    color_warp = out_img
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    pts_left = np.array([np.transpose(np.vstack([lineLeft.poly_plotx, lineLeft.poly_ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([lineRight.poly_plotx, lineRight.poly_ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
     
    #Draw the lane onto the warped blank image
    start = time()
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    
    newwarp = cv2.warpPerspective(color_warp, M, (img.shape[1], img.shape[0])) 
    result = cv2.addWeighted(img, 1, newwarp, 0.8, 0)
    
    end = time()
    print(f'Draw the lane {end - start} seconds!')
    
    ### Anotate Radius of curvature
    start = time()
    diff, mean, text = checkRadius(lineLeft, lineRight )

    result_annotated = cv2.putText(result, text, org= (50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                   fontScale=2, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
    
    end = time()
    print(f'Radius of curvature {end - start} seconds!')
    ### Anotate Vehicle position
    start = time()
    dev, text = calculateDeviation(result, lineLeft,lineRight,)

    result_annotated = cv2.putText(result, text, org= (50, 100), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                       fontScale=2, color= (255, 255, 255), thickness=2, lineType=cv2.LINE_AA)
      
    end = time()
    print(f'Vehicle position {end - start} seconds!')   
    #out = np.dstack((out_img*255, out_img*255, out_img*255))
    
    return result_annotated

In [3]:
# Instanciate cLine classes
lineLeft = Line()
lineRight = Line()


white_output = 'output_videos/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,1)
#clip1 = VideoFileClip("test_videos/project_video.mp4")
white_clip = clip1.fl_image(process_frame_profiling) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

resizeImage took 0.0 seconds!
undistorted took 0.023970365524291992 seconds!
colorEnhancement took 0.009995222091674805 seconds!
grayscale took 0.0009992122650146484 seconds!
gaussian_blur took 0.00099945068359375 seconds!
sobel_thresh took 0.0997314453125 seconds!
region_of_interest took 0.0 seconds!
warp_image took 0.0 seconds!
find_lane_pixels took 0.027791738510131836 seconds!
Search blindly image 0.012993335723876953 seconds!


t:   0%|                                                                              | 0/25 [00:00<?, ?it/s, now=None]

Draw the lane 0.0059967041015625 seconds!
Radius of curvature 0.0019981861114501953 seconds!
Vehicle position 0.0021066665649414062 seconds!
Moviepy - Building video output_videos/solidWhiteRight.mp4.
Moviepy - Writing video output_videos/solidWhiteRight.mp4

resizeImage took 0.0 seconds!
undistorted took 0.022987842559814453 seconds!
colorEnhancement took 0.014991521835327148 seconds!
grayscale took 0.0009999275207519531 seconds!
gaussian_blur took 0.0009984970092773438 seconds!
sobel_thresh took 0.1495044231414795 seconds!
region_of_interest took 0.0009996891021728516 seconds!
warp_image took 0.003997087478637695 seconds!
Search based on coefs 0.01399087905883789 seconds!
Draw the lane 0.009992837905883789 seconds!
Radius of curvature 0.001993894577026367 seconds!
Vehicle position 0.0010023117065429688 seconds!


t:   8%|█████▌                                                                | 2/25 [00:01<00:18,  1.25it/s, now=None]

resizeImage took 0.0 seconds!
undistorted took 0.02999281883239746 seconds!
colorEnhancement took 0.009887933731079102 seconds!
grayscale took 0.0019969940185546875 seconds!
gaussian_blur took 0.0009965896606445312 seconds!
sobel_thresh took 0.10873818397521973 seconds!
region_of_interest took 0.0 seconds!
warp_image took 0.0 seconds!
Search based on coefs 0.01322031021118164 seconds!
Draw the lane 0.005995988845825195 seconds!


...

sobel_thresh took 0.14391875267028809 seconds!
region_of_interest took 0.0010008811950683594 seconds!
warp_image took 0.002995729446411133 seconds!
Search based on coefs 0.013991355895996094 seconds!
Draw the lane 0.006986856460571289 seconds!
Radius of curvature 0.0009989738464355469 seconds!
Vehicle position 0.0010004043579101562 seconds!
resizeImage took 0.0 seconds!
undistorted took 0.022986888885498047 seconds!
colorEnhancement took 0.00999593734741211 seconds!
grayscale took 0.0009987354278564453 seconds!
gaussian_blur took 0.0009987354278564453 seconds!


t: 100%|█████████████████████████████████████████████████████████████████████| 25/25 [00:07<00:00,  3.96it/s, now=None]

sobel_thresh took 0.15291428565979004 seconds!
region_of_interest took 0.0009987354278564453 seconds!
warp_image took 0.0029976367950439453 seconds!
Search based on coefs 0.013990640640258789 seconds!
Draw the lane 0.005997180938720703 seconds!
Radius of curvature 0.0009984970092773438 seconds!
Vehicle position 0.00099945068359375 seconds!
resizeImage took 0.0 seconds!
undistorted took 0.02098679542541504 seconds!
colorEnhancement took 0.012996673583984375 seconds!
grayscale took 0.0009982585906982422 seconds!
gaussian_blur took 0.0 seconds!


sobel_thresh took 0.13092470169067383 seconds!
region_of_interest took 0.0009989738464355469 seconds!
warp_image took 0.003998279571533203 seconds!
Search based on coefs 0.01698923110961914 seconds!
Draw the lane 0.005995988845825195 seconds!
Radius of curvature 0.0009996891021728516 seconds!
Vehicle position 0.0009999275207519531 seconds!


Moviepy - Done !
Moviepy - video ready output_videos/solidWhiteRight.mp4
Wall time: 8.42 s


In [4]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))